In [1]:
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

In [2]:
from model import Unet
from hyperparameters import *
from utils import save_checkpoint, get_loaders, check_accuracy, save_predictions_as_imgs

In [3]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.float().to(device = DEVICE)
        targets = targets.float().to(device = DEVICE)

        # Forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
        
        # Backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Update Loop
        loop.set_postfix(loss = loss.item())

In [4]:
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    #transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0])
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_transform_gt = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    #transforms.Normalize(mean=[0.0], std=[1.0])
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    #transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0])
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform_gt = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    #transforms.Normalize(mean=[0.0], std=[1.0])
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [5]:
model = Unet(in_channels = IN_CHANNELS, out_channels = OUT_CHANNELS).to(device = DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

train_loader, test_loader = get_loaders(train_transform, test_transform, train_transform_gt, test_transform_gt)

In [6]:
def main():
    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

    checkpoint = {
        "state_dict" : model.state_dict(),
        "optimizer" : optimizer.state_dict()
    }
    
    save_checkpoint(checkpoint)

    check_accuracy(test_loader, model, device = DEVICE)

    save_predictions_as_imgs(test_loader, model, folder = "saved_images/", device = DEVICE)

In [7]:
if __name__ == "__main__":
    main()

C:\Users\andre\AppData\Local\Temp\ipykernel_22776\3715779838.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
c:\Users\andre\miniconda3\envs\pytorch\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\andre\AppData\Local\Temp\ipykernel_22776\3815435854.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
c:\Users\andre\miniconda3\envs\pytorch\Lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 10/10 [00:59<00:00,  5.98s/it, loss=-1.61e+4]


=> Saving checkpoint
Accuracy: 0.60%, Dice Score: 1.9994
